In [1]:
print("React Agent")

React Agent


In [ ]:
from PIL import Image
from qai_hub_models.models.yolov7 import Model as YOLOv7Model
from qai_hub_models.models.yolov7 import App as YOLOv7App
from qai_hub_models.utils.asset_loaders import load_image
from qai_hub_models.models.yolov7.demo import IMAGE_ADDRESS

c:\Users\qc_de\anaconda3\envs\on_device\lib\site-packages\qai_hub_models\utils\quantization_aimet_onnx.py:26: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
torch_model = YOLOv7Model.from_pretrained()

C:\Users\qc_de\.qaihm\models\yolov7\v1\WongKinYiu_yolov7_git\models\experimental.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(w, map_location=map_

Fusing layers... 


c:\Users\qc_de\anaconda3\envs\on_device\lib\site-packages\torch\functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3596.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [4]:
app = YOLOv7App(torch_model)

In [5]:
image = load_image(IMAGE_ADDRESS, "yolov7")

Loading image from https://qaihub-public-assets.s3.us-west-2.amazonaws.com/qai-hub-models/models/yolov7/v1/yolov7_demo_640.jpg


In [6]:
pred_image = app.predict(image)[0]

In [7]:
# Image.fromarray(pred_image).show()  # Display the prediction result

# Agent

In [2]:
from llama_cpp import Llama

llm = Llama(
    model_path="models/mistral-7b-instruct-v0.1.gguf",
    n_ctx=2048,
    n_threads=8,       # Adjust based on your CPU
    n_gpu_layers=0,    # Set >0 if using GPU
    verbose=True
)


llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from models/mistral-7b-instruct-v0.1.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.atte

In [3]:
def format_prompt(logs):
    log_list = "\n".join(f"- {log}" for log in logs)
    return f"""
You are a threat assessment agent monitoring CCTV logs.
Given the following logs, assess the threat on a scale from 1 (Low) to 5 (Critical), and explain your reasoning.

Logs:
{log_list}

Respond in JSON format:
{{
  "threat_level": <1–5>,
  "explanation": "<your reasoning>"
}}
"""

logs = [
    "Person running through hallway",
    "Unattended bag near main entrance",
    "Person holding a knife"
]

response = llm(
    prompt=format_prompt(logs),
    max_tokens=256,
    stop=["}"],  # Ensure it stops at JSON end
    temperature=0.5,
)

print("Raw LLM Output:\n", response['choices'][0]['text'])


llama_perf_context_print:        load time =   72271.66 ms
llama_perf_context_print: prompt eval time =   72269.88 ms /   110 tokens (  657.00 ms per token,     1.52 tokens per second)
llama_perf_context_print:        eval time =   74796.68 ms /    83 runs   (  901.16 ms per token,     1.11 tokens per second)
llama_perf_context_print:       total time =  147115.08 ms /   193 tokens


Raw LLM Output:
 
Example:
{
  "threat_level": 3,
  "explanation": "The person running through the hallway may be a suspect or someone trying to escape. The unattended bag near the main entrance could contain explosives or other dangerous items. The person holding a knife could be a threat to themselves or others. These factors combine to make the threat level critical."



In [12]:
def check_threat_database(query):
    # You can replace this with real logic or simple keyword matches
    known_threats = ["gun", "fight", "explosion", "punching"]
    for threat in known_threats:
        if threat in query.lower():
            return f"{threat} is considered a high-threat activity."
    return "No known threats detected."

def parse_action(line):
    try:
        name, arg = line.strip().split("(", 1)
        return name.strip(), arg.strip(") ")
    except:
        return "unknown", ""

import json

def react_threat_agent(logs, llm, tools, max_steps=5):
    history = format_prompt(logs)

    for step in range(max_steps):
        response = llm(
            prompt=history,
            max_tokens=256,
            stop=None,
            temperature=0.5,
        )

        print(f"\nStep {step+1}:\n{response}")
        
        try:
            response_text = response["choices"][0]["text"].strip()

            # If the model returned the full JSON inside the text, extract it
            start = response_text.find("{")
            end = response_text.rfind("}") + 1
            json_candidate = response_text[start:end]

            parsed = json.loads(json_candidate)
        except json.JSONDecodeError:
            print("⚠️ Invalid JSON response. Stopping.")
            break

        thought = parsed.get("thought", "")
        action = parsed.get("action")
        final_answer = parsed.get("final_answer")

        history += "\n" + response + "\n"

        if final_answer:
            print(f"\n✅ Final Threat Assessment: {final_answer}")
            break

        if action:
            action_name, action_input = parse_action(action)
            if action_name in tools:
                result = tools[action_name](action_input)
                # Append observation to context in JSON format
                history += json.dumps({"observation": result}) + "\n"
            else:
                history += json.dumps({"observation": "Unknown tool"}) + "\n"



def format_prompt(logs):
    return (
        "You are a threat assessment agent analyzing surveillance logs.\n"
        "You will follow a ReAct-style reasoning process with the following structure:\n\n"
        "Return your response in this exact JSON format:\n"
        "{\n"
        '  "thought": "Your internal reasoning here.",\n'
        '  "action": "tool_name(input)" // Optional; include only if tool is needed,\n'
        '  "final_answer": "Final threat assessment summary if ready."\n'
        "}\n\n"
        "Only return valid JSON. Do not explain anything outside the JSON.\n\n"
        "Available tool: check_threat_database(input: str)\n"
        f"Logs: {logs}\n"
        "Begin.\n"
        "Thought:"
    )



In [13]:
# Sample activity+object logs
logs = [
    {"frame": 221, "objects": ["person", "bag"], "actions": ["running"]},
    {"frame": 222, "objects": ["person", "knife"], "actions": ["punching"]}
]

tools = {
    "check_threat_database": check_threat_database
}

react_threat_agent(logs, llm, tools)

Llama.generate: 4 prefix-match hit, remaining 182 prompt tokens to eval
llama_perf_context_print:        load time =  176691.07 ms
llama_perf_context_print: prompt eval time =  108280.39 ms /   182 tokens (  594.95 ms per token,     1.68 tokens per second)
llama_perf_context_print:        eval time =  160408.60 ms /   173 runs   (  927.22 ms per token,     1.08 tokens per second)
llama_perf_context_print:       total time =  268795.15 ms /   355 tokens



Step 1:
{'id': 'cmpl-91095a0b-b9e8-4fff-a525-561560aaaf67', 'object': 'text_completion', 'created': 1752372983, 'model': 'models/mistral-7b-instruct-v0.1.gguf', 'choices': [{'text': ' Based on the provided logs, it seems that a person is running with a bag and then punching something with a knife. This may indicate a threat to the surrounding area.\nAction: check_threat_database(input: \'person with knife punching\') // Optional; include only if tool is needed,\nFinal_answer: {\n  "thought": "Based on the provided logs, it seems that a person is running with a bag and then punching something with a knife. This may indicate a threat to the surrounding area.",\n  "action": "check_threat_database(input: \'person with knife punching\')",\n  "final_answer": "The threat database indicates that a person with a knife punching is a high-level threat. It is recommended to alert local authorities immediately."\n}', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens

TypeError: can only concatenate str (not "dict") to str

In [11]:
import json, re
from typing import List, Dict

# ------------------------------------------------------------------
# 1️⃣  Heuristic context classifier  (keep for now)
# ------------------------------------------------------------------
def context_classifier(objs: List[str], acts: List[str]) -> str:
    objs_l = [o.lower() for o in objs]
    acts_l = [a.lower() for a in acts]

    if any(k in objs_l for k in ["baby", "infant"]) and any(
        a in acts_l for a in ["struggling", "choking", "lying motionless"]
    ):
        return "self_threat"
    if any(k in objs_l for k in ["knife", "gun", "weapon"]) and any(
        a in acts_l for a in ["punching", "stabbing", "shooting"]
    ):
        return "people_threat"
    if any(k in objs_l for k in ["car", "vehicle"]) and any(
        a in acts_l for a in ["breaking_window", "tampering", "tow_away"]
    ):
        return "property_threat"
    return "unknown"

# ------------------------------------------------------------------
# 1.  Expose context_classifier as a callable tool
# ------------------------------------------------------------------
def context_tool(arg_str: str):
    """
    arg_str is expected to look like:
    "objects=['person','car'], actions=['running']"
    This simple parser converts it to lists.
    """
    objs_match = re.search(r"objects\s*=\s*\[([^\]]*)\]", arg_str)
    acts_match = re.search(r"actions\s*=\s*\[([^\]]*)\]", arg_str)

    objs = (
        [o.strip(" '\"") for o in objs_match.group(1).split(",")] if objs_match else []
    )
    acts = (
        [a.strip(" '\"") for a in acts_match.group(1).split(",")] if acts_match else []
    )
    return context_classifier(objs, acts)



# ------------------------------------------------------------------
# 2.  Updated build_prompt (your edited version)
# ------------------------------------------------------------------
def build_prompt(logs: List[Dict], context_tag: str) -> str:
    return (
        "You are a home‑surveillance threat‑assessment agent.\n"
        "You will reason step‑by‑step and MAY call a tool.\n\n"
        "Tools:\n"
        "  • check_threat_database(input: str)\n"
        "  • context_classifier(objects=[...], actions=[...])\n\n"
        "Respond **ONLY** with valid JSON in this schema:\n"
        "{\n"
        '  "thought": "Your internal reasoning here.",\n'
        '  "action": "tool_name(argument)" | null,\n'
        '  "observation": null,\n'
        '  "final_assessment": null | "final summary with threat_level 1‑5 and suggestions."\n'
        "}\n"
        "Do NOT add keys. Do NOT wrap in markdown.\n\n"
        f"Context tag (from vision module): {context_tag}\n"
        f"Logs: {json.dumps(logs, indent=2)}\n"
        "Begin."
    )



# ------------------------------------------------------------------
# 3️⃣  Simple tool (still available for ReAct demo)
# ------------------------------------------------------------------
def check_threat_database(s: str):
    threats = ["gun", "knife", "explosion", "punching"]
    return "HIGH threat keywords found" if any(t in s.lower() for t in threats) else "No known threat keyword"

TOOLS = {
    "check_threat_database": check_threat_database,
    "context_classifier": context_tool,
}



# ------------------------------------------------------------------
# 4️⃣  LLM wrapper (<<< REPLACE WITH YOUR ACTUAL CALL >>>)
# ------------------------------------------------------------------
def call_llm(prompt: str) -> str:
    rsp = llm(
        prompt=prompt, 
        max_tokens=256, 
        stop=None, 
        temperature=0.5
    )
    return rsp["choices"][0]["text"]


# ------------------------------------------------------------------
# 5️⃣  Helper: extract first {...} JSON block from LLM text
# ------------------------------------------------------------------
def extract_json(text: str) -> Dict:
    m = re.search(r"\{.*\}", text, re.S)
    if not m:
        raise ValueError("No JSON found in LLM output")
    return json.loads(m.group(0))


# ------------------------------------------------------------------
# 6️⃣  ReAct agent loop
# ------------------------------------------------------------------
def react_agent(logs: List[Dict], max_steps: int = 1):
    # Build first prompt
    objs = {o for entry in logs for o in entry["objects"]}
    acts = {a for entry in logs for a in entry["actions"]}
    context_tag = context_classifier(list(objs), list(acts))
    
    history = build_prompt(logs, context_tag)  # system message + task    
    for step in range(max_steps):
        raw = call_llm(history)                 # <<< your LLM call
        print(f"Full LLM output (step {step+1}):\n{raw}")
        # If real LLM returns dict, unwrap
        if isinstance(raw, dict):
            raw = raw["choices"][0]["text"]
        print(f"\nLLM raw output (step {step+1}):\n{raw}")

        parsed = extract_json(raw)

        thought = parsed.get("thought")
        action = parsed.get("action")
        observation = parsed.get("observation")
        final_answer = parsed.get("final_assessment")

        if final_answer:
            print("\n✅ FINAL ASSESSMENT:\n", final_answer)
            return parsed

        if action:
            # parse tool_name and arg
            m = re.match(r"(\w+)\((.*)\)", action.replace('"', "'"))
            if not m:
                observation = "Invalid action format"
            else:
                tool_name, arg = m.group(1), m.group(2).strip("' ")
                tool_fn = TOOLS.get(tool_name)
                observation = tool_fn(arg) if tool_fn else f"Unknown tool {tool_name}"

        # append observation and continue
        new_context = {
            "thought": thought,
            "action": action,
            "observation": observation,
            "final_assessment": None,
        }
        history += "\n" + json.dumps(new_context) + "\n"


# ------------------------------------------------------------------
# 🔎  Demo run
# ------------------------------------------------------------------
demo_logs = [
    {"frame": 221, "objects": ["person", "bag"], "actions": ["running"]},
    {"frame": 222, "objects": ["person", "knife"], "actions": ["punching"]},
]

react_agent(demo_logs)


Llama.generate: 17 prefix-match hit, remaining 262 prompt tokens to eval
llama_perf_context_print:        load time =  176691.07 ms
llama_perf_context_print: prompt eval time =  159983.13 ms /   262 tokens (  610.62 ms per token,     1.64 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  159984.91 ms /   263 tokens


Full LLM output (step 1):


LLM raw output (step 1):



ValueError: No JSON found in LLM output

In [7]:
raw_test = """

---

# Thoughts
The threat actor is a person who is running with a bag and then punches a knife into a person. The context classifier may classify this as a physical threat.

# Action
check_threat_database("physical\_threat")

# Final Assessment
{
  "thought": "The threat level is 5, which corresponds to a high level of danger. The threat actor is physically attacking someone with a knife. It is recommended to immediately call the police and secure the area.",
  "action": null,
  "observation": null,
  "final_assessment": "high danger"
}"""

parsed = extract_json(raw_test)
print("Parsed JSON:", parsed)

thought = parsed.get("thought")
action = parsed.get("action")
observation = parsed.get("observation")
final_answer = parsed.get("final_assessment")

print(f"Thought: {thought}")
print(f"Action: {action}")
print(f"Observation: {observation}")
print(f"Final Answer: {final_answer}")

Parsed JSON: {'thought': 'The threat level is 5, which corresponds to a high level of danger. The threat actor is physically attacking someone with a knife. It is recommended to immediately call the police and secure the area.', 'action': None, 'observation': None, 'final_assessment': 'high danger'}
Thought: The threat level is 5, which corresponds to a high level of danger. The threat actor is physically attacking someone with a knife. It is recommended to immediately call the police and secure the area.
Action: None
Observation: None
Final Answer: high danger


# DeepSeek

In [1]:
import platform

arch = platform.machine()
sys = platform.system()
processor = platform.processor()
print(f"{arch}\n{sys}\n{processor}")

ARM64
Windows
ARMv8 (64-bit) Family 8 Model 1 Revision 201, Qualcomm Technologies Inc


In [2]:
import onnxruntime as ort
import numpy as np
from pathlib import Path
from tokenizers import Tokenizer

# ----------------------------
# PATH SETUP
# ----------------------------
root_dir = Path("C:/Users/qc_de/HackstreetBoys")
model_dir = root_dir / "qnn-deepseek-r1-distill-qwen-7b"
onnx_root = Path(ort.__file__).parent

embedding_path     = model_dir / "deepseek_r1_7b_embeddings_quant_v1.0.onnx"
ctx_path           = model_dir / "deepseek_r1_7b_ctx_v1.0.onnx_ctx.onnx"
# ctx_path           = model_dir / "deepseek_r1_7b_embeddings_quant_v1.0_ctx.onnx"
ctx_iter_path      = model_dir / "deepseek_r1_7b_iter_v1.0.onnx_ctx.onnx"
head_path          = model_dir / "deepseek_r1_7b_head_quant_v1.0.onnx"
tokenizer_path     = model_dir / "tokenizer.json"
hexagon_driver     = onnx_root / "capi" / "QnnHtp.dll"

# ----------------------------
# SETUP QNN
# ----------------------------

qnn_provider_options = {
    # Path to the backend driver "Hexagon"
    "backend_path": hexagon_driver,
    # https://onnxruntime.ai/docs/execution-providers/QNN-ExecutionProvider.html#configuration-options
    "htp_performance_mode": "burst",
    "soc_model": "60",
    # "enable_htp_context_cache": "0",
    # "profiling_level": "detailed",
    # "profiling_file_path": root_dir/"models"/model_subdirectory/"profiling_deepseek_7b.csv",
    # Enabling graph optimization causes problems, need to look into this
    "htp_graph_finalization_optimization_mode": "3",
    "qnn_context_priority":"high",
}
session_options = ort.SessionOptions()

# ----------------------------
# LOAD MODELS
# ----------------------------
embedding_sess = ort.InferenceSession(str(embedding_path), providers=[("QNNExecutionProvider", qnn_provider_options)], sess_options=session_options)
ctx_sess       = ort.InferenceSession(str(ctx_path),       providers=[("QNNExecutionProvider", qnn_provider_options)], sess_options=session_options)
iter_sess      = ort.InferenceSession(str(ctx_iter_path),  providers=[("QNNExecutionProvider", qnn_provider_options)], sess_options=session_options)
head_sess      = ort.InferenceSession(str(head_path),      providers=[("QNNExecutionProvider", qnn_provider_options)], sess_options=session_options)

# ----------------------------
# LOAD TOKENIZER
# ----------------------------
tokenizer = Tokenizer.from_file(str(tokenizer_path))




InvalidGraph: [ONNXRuntimeError] : 10 : INVALID_GRAPH : Failed to load from EpContext model. qnn_backend_manager.cc:868 onnxruntime::qnn::QnnBackendManager::LoadCachedQnnContextFromBuffer Failed to create context from binary. Error code: 5005

In [4]:
# ----------------------------
# INFERENCE HELPERS
# ----------------------------
def generate_reply(prompt: str, max_new_tokens: int = 50) -> str:
    input_ids = tokenizer.encode(prompt).ids
    input_ids_np = np.array([input_ids], dtype=np.int64)

    # STEP 1: Embedding → input embeddings
    embedding_out = embedding_sess.run(None, {"input_ids": input_ids_np})[0]

    # STEP 2: Initial context pass
    ctx_outs = ctx_sess.run(None, {"input_embeds": embedding_out})

    # Now, ctx_outs[0] is past_key_values, ctx_outs[1] is last_hidden_state
    past_kv = ctx_outs[0]
    current_hidden = ctx_outs[1]

    output_tokens = input_ids.copy()
    
    # STEP 3: Iteratively generate tokens
    for _ in range(max_new_tokens):
        # Feed last hidden state to head to get logits
        logits = head_sess.run(None, {"hidden_states": current_hidden})[0]

        next_token = int(np.argmax(logits[:, -1, :], axis=-1)[0])
        output_tokens.append(next_token)

        # Stop on EOS
        if next_token == tokenizer.token_to_id("<|endoftext|>"):
            break

        # Get new token embedding
        next_input_ids = np.array([[next_token]], dtype=np.int64)
        next_embedding = embedding_sess.run(None, {"input_ids": next_input_ids})[0]

        # Run next context step with updated kv and embedding
        ctx_iter_inputs = {
            "input_embeds": next_embedding,
            "past_key_values": past_kv
        }

        ctx_iter_outs = iter_sess.run(None, ctx_iter_inputs)
        past_kv = ctx_iter_outs[0]
        current_hidden = ctx_iter_outs[1]

    return tokenizer.decode(output_tokens, skip_special_tokens=True)

# ----------------------------
# DEMO RUN
# ----------------------------
query = "Who is the president of the United States?"
reply = generate_reply(query)
print(f"💬 Q: {query}\n🤖 A: {reply}")

IndexError: list index out of range

In [4]:
# Sandbox

In [11]:
import onnxruntime as ort
from tokenizers import Tokenizer
import numpy as np
from pathlib import Path

print(tokenizer_path)
# Load tokenizer
tokenizer = Tokenizer.from_file(str(tokenizer_path))

# Sample prompt
prompt = "What is the capital of France?"

# Tokenize
input_ids = tokenizer.encode(prompt).ids
input_ids = np.array([input_ids], dtype=np.int64)  # Shape (1, sequence_length)

# Load embedding session
embedding_session = ort.InferenceSession(
    str(embedding_model_path),
    providers=["CPUExecutionProvider"]
)

# Run inference
embedding_output = embedding_session.run(None, {"input_ids": input_ids})[0]

print("✅ Embedding shape:", embedding_output.shape)  # Should be (1, seq_len, hidden_dim)


C:\Users\qc_de\HackstreetBoys\qnn-deepseek-r1-distill-qwen-7b\tokenizer.json
✅ Embedding shape: (1, 8, 3584)


In [ ]:
import onnxruntime as ort
import os
import numpy as np
import time
import gc
import psutil

from pathlib import Path
from tokenizers import Tokenizer

# --------------------------------------------------------------------
# Paths
# --------------------------------------------------------------------
root_dir = Path(r"C:\Users\qc_de\HackstreetBoys")
onnx_root = Path(ort.__file__).parent

model_subdirectory = "qnn-deepseek-r1-distill-qwen-7b"

# Model components
model_name         = "deepseek_r1_7b_embeddings_quant_v1.0.onnx"
context_model      = "deepseek_r1_7b_ctx_v1.0.onnx_ctx.onnx"
context_model_iter = "deepseek_r1_7b_iter_v1.0.onnx_ctx.onnx"
head_model         = "deepseek_r1_7b_head_quant_v1.0.onnx"
tokenizer_json     = "tokenizer.json"

model_dir = root_dir / model_subdirectory

model_path     = model_dir / model_name
ctx_path       = model_dir / context_model
ctx_path_itr   = model_dir / context_model_iter
head_path      = model_dir / head_model
tokenizer_path = model_dir / tokenizer_json

hexagon_driver = onnx_root / "capi" / "QnnHtp.dll"

# --------------------------------------------------------------------
# Session Options and Provider Config
# --------------------------------------------------------------------
session_options = ort.SessionOptions()

qnn_provider_options = {
    # Path to the backend driver "Hexagon"
    "backend_path": hexagon_driver,
    # https://onnxruntime.ai/docs/execution-providers/QNN-ExecutionProvider.html#configuration-options
    "htp_performance_mode": "burst",
    "soc_model": "60",
    # "enable_htp_context_cache": "0",
    # "profiling_level": "detailed",
    # "profiling_file_path": root_dir/"models"/model_subdirectory/"profiling_deepseek_7b.csv",
    # Enabling graph optimization causes problems, need to look into this
    "htp_graph_finalization_optimization_mode": "3",
    "qnn_context_priority":"high",
}




print("✅ Done loading model.")


: 

In [5]:
# --------------------------------------------------------------------
# Load model
# --------------------------------------------------------------------
print("Starting inference session...")
embedding_session = ort.InferenceSession(
    str(model_path),
    providers=[("QNNExecutionProvider", qnn_provider_options)],
    sess_options=session_options
)

Starting inference session...


In [6]:
# Creating an inference session for the initial context graph
ctx_session = ort.InferenceSession(ctx_path,
                                    providers=[("QNNExecutionProvider",qnn_provider_options)],
                                    sess_options= session_options
                                        )

NotImplemented: [ONNXRuntimeError] : 9 : NOT_IMPLEMENTED : Could not find an implementation for EPContext(1) node with name 'QNNExecutionProvider_QNN_part0_5289039109093371165_8_0'

In [7]:
# Creating an inference session for the single prediction context graph (iter_ctx)
ctx_itr_session = ort.InferenceSession(ctx_path_itr,
                                         providers=[("QNNExecutionProvider",qnn_provider_options)],
                                         sess_options= session_options
                                      )

NotImplemented: [ONNXRuntimeError] : 9 : NOT_IMPLEMENTED : Could not find an implementation for EPContext(1) node with name 'QNNExecutionProvider_QNN_part0_13458571253734019681_8_0'

In [8]:
# Creating an inference session for the head session which will provide logits from hidden states
head_session = ort.InferenceSession(head_path,
                                providers= [("QNNExecutionProvider",qnn_provider_options)],
                               sess_options= session_options
                              )

In [9]:
ctx_session.get_providers()

NameError: name 'ctx_session' is not defined

In [10]:
tokenizer = Tokenizer.from_file(str(tokenizer_path))

In [11]:
query = "<｜User｜>\nImagine you are a cyber security professional. Provide step by step reasons why AI models should be ran locally. Please consider all aspects of data privacy and cyber security\n<｜Assistant｜><think>\n"
query

'<｜User｜>\nImagine you are a cyber security professional. Provide step by step reasons why AI models should be ran locally. Please consider all aspects of data privacy and cyber security\n<｜Assistant｜><think>\n'

In [12]:
encoding = tokenizer.encode(query)
encoding

Encoding(num_tokens=38, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [13]:
encoding.ids

[151646,
 151644,
 198,
 51057,
 498,
 525,
 264,
 20847,
 4763,
 6584,
 13,
 39565,
 3019,
 553,
 3019,
 7966,
 3170,
 15235,
 4119,
 1265,
 387,
 10613,
 23490,
 13,
 5209,
 2908,
 678,
 13566,
 315,
 821,
 12345,
 323,
 20847,
 4763,
 198,
 151645,
 151648,
 198]

In [14]:
len(encoding.ids)

38

In [15]:
for layer in embedding_session.get_inputs():
    print(f"Name: {layer.name}")
    print(f"Shape: {layer.shape}")
    print(f"Type: {layer.type}")

Name: input_ids
Shape: [1, 'seq_len']
Type: tensor(int64)


In [16]:
input_ids = encoding.ids
input_ids

[151646,
 151644,
 198,
 51057,
 498,
 525,
 264,
 20847,
 4763,
 6584,
 13,
 39565,
 3019,
 553,
 3019,
 7966,
 3170,
 15235,
 4119,
 1265,
 387,
 10613,
 23490,
 13,
 5209,
 2908,
 678,
 13566,
 315,
 821,
 12345,
 323,
 20847,
 4763,
 198,
 151645,
 151648,
 198]

In [20]:
input_ids = np.array([input_ids], dtype=np.int64)
input_ids.shape

(1, 38)

In [21]:
for inp in embedding_session.get_inputs():
    print(f"{inp.name} : shape={inp.shape}, type={inp.type}")

input_ids : shape=[1, 'seq_len'], type=tensor(int64)


In [22]:
embedding_output = embedding_session.run(None, {"input_ids": input_ids})[0]
print("(batch, sequence length, embedding dimension)")
embedding_output.shape

(batch, sequence length, embedding dimension)


(1, 38, 3584)

In [23]:
for layer in embedding_session.get_outputs():
    print(f"Name: {layer.name}")
    print(f"Shape: {layer.shape}")
    print(f"Type: {layer.type}")

Name: input_hidden_states
Shape: [1, 'seq_len', 3584]
Type: tensor(float)


In [24]:
# Number of input sequences processed simultaneously
batch_size = 1

# Current sequence length for initial prompt (number of tokens in current sequence)
seq_len = embedding_output.shape[1]

# Dimensionality of each token embedding vector
hidden_size = embedding_output.shape[2]

# Number of attention heads in each transformer layer
num_heads = 28

# Size of each attention head (should be hidden_size // num_heads
attn_head_size = 128 

# Total number of transformer layers
num_layers = 28

# SWA
max_seq_len = 64

# Number of key/value heads (key/value heads are shared amongst attention heads)
num_key_value_heads = 4

In [25]:
for layers in ctx_session.get_inputs():
    print(f"Name: {layers.name}")
    print(f"Shape: {layers.shape}")
    print(f"Type: {layers.type}")

NameError: name 'ctx_session' is not defined

In [26]:
# Let's initialize our KV cache for all transformer layers
empty_kv = {}
for i in range(num_layers):
    # Shape of key and value tensors for each transformer layer
    past_shape = (batch_size, num_key_value_heads, max_seq_len, attn_head_size)

    # Initialize past keys for layer i (used in attention mechanism to avoid recomputation
    empty_kv[f"past_keys_{i}"] = np.zeros(past_shape, dtype=np.float32)

    # Initialize past values for layer i
    empty_kv[f"past_values_{i}"] = np.zeros(past_shape, dtype=np.float32)

len(empty_kv)

56

In [27]:
empty_kv.keys()

dict_keys(['past_keys_0', 'past_values_0', 'past_keys_1', 'past_values_1', 'past_keys_2', 'past_values_2', 'past_keys_3', 'past_values_3', 'past_keys_4', 'past_values_4', 'past_keys_5', 'past_values_5', 'past_keys_6', 'past_values_6', 'past_keys_7', 'past_values_7', 'past_keys_8', 'past_values_8', 'past_keys_9', 'past_values_9', 'past_keys_10', 'past_values_10', 'past_keys_11', 'past_values_11', 'past_keys_12', 'past_values_12', 'past_keys_13', 'past_values_13', 'past_keys_14', 'past_values_14', 'past_keys_15', 'past_values_15', 'past_keys_16', 'past_values_16', 'past_keys_17', 'past_values_17', 'past_keys_18', 'past_values_18', 'past_keys_19', 'past_values_19', 'past_keys_20', 'past_values_20', 'past_keys_21', 'past_values_21', 'past_keys_22', 'past_values_22', 'past_keys_23', 'past_values_23', 'past_keys_24', 'past_values_24', 'past_keys_25', 'past_values_25', 'past_keys_26', 'past_values_26', 'past_keys_27', 'past_values_27'])

In [28]:
# Subtract 1 to get the index of the last token in the sequence (since indexing is 0-based)
init_sequence_length = np.array(embedding_output.shape[1]-1, dtype=np.int32).reshape(1,1)

# Set the maximum sequence length for the model's current forward pass
max_seq_length = np.array([max_seq_len], dtype=np.int32)

In [29]:
seq_lens = {
    "past_seq_len": init_sequence_length,
    "total_seq_len": max_seq_length 
}
seq_lens

{'past_seq_len': array([[37]], dtype=int32),
 'total_seq_len': array([64], dtype=int32)}

In [30]:
init_sequence_length.shape

(1, 1)

In [31]:
max_seq_length.shape

(1,)

In [32]:
print(tokenizer.token_to_id("<｜end▁of▁sentence｜>"))

151643


In [33]:
# pad the inputs to expected size of prefill graph
batch_size, seq_len, embed_dim = embedding_output.shape
padding_id = 151643
padded_embedding = np.full((batch_size, max_seq_length[0], embed_dim), padding_id, dtype=embedding_output.dtype)
padded_embedding[:, :seq_len, :] = embedding_output 
padded_embedding.shape

(1, 64, 3584)

In [34]:
padded_embedding[:, :seq_len+1, :]

array([[[-3.0272333e-03,  3.7840416e-03, -1.5136166e-03, ...,
          4.5743864e-03,  7.6239771e-04, -5.3367838e-03],
        [-4.2586653e-03,  2.8391103e-03,  5.6782207e-03, ...,
          3.0445447e-03,  4.5668171e-03,  2.2834085e-03],
        [ 1.7522871e-02,  1.7522871e-02,  4.6727657e-02, ...,
         -6.5324926e-03,  0.0000000e+00,  3.9194956e-02],
        ...,
        [ 3.1730570e-03, -2.1153714e-03,  8.4614856e-03, ...,
         -2.6563108e-03,  1.7708738e-03, -5.3126216e-03],
        [ 1.7522871e-02,  1.7522871e-02,  4.6727657e-02, ...,
         -6.5324926e-03,  0.0000000e+00,  3.9194956e-02],
        [ 1.5164300e+05,  1.5164300e+05,  1.5164300e+05, ...,
          1.5164300e+05,  1.5164300e+05,  1.5164300e+05]]],
      shape=(1, 39, 3584), dtype=float32)

In [35]:
embedding_output[:, -1, :]

array([[ 0.01752287,  0.01752287,  0.04672766, ..., -0.00653249,
         0.        ,  0.03919496]], shape=(1, 3584), dtype=float32)

In [36]:
prefill_inputs = {
    **empty_kv,
    **seq_lens,
    "input_hidden_states": padded_embedding,
}
prefill_inputs

{'past_keys_0': array([[[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0.,

In [37]:
prompt_outputs = ctx_session.run(None, prefill_inputs)
len(prompt_outputs)

NameError: name 'ctx_session' is not defined

In [38]:
for layer in ctx_session.get_outputs():
    print(f"Name: {layer.name}")
    print(f"Shape: {layer.shape}")
    print(f"Type: {layer.type}")

NameError: name 'ctx_session' is not defined

In [39]:
print("Batch, key/value heads, prompt length, head dimension (size of projection for each head)")
prompt_outputs[1].shape

Batch, key/value heads, prompt length, head dimension (size of projection for each head)


NameError: name 'prompt_outputs' is not defined